In [1]:
import pandas as pd
import sklearn
import numpy as np


In [2]:
data = pd.read_csv("out.csv").dropna()

# Modify the 'winner' column based on conditions
data['winner'] = data.apply(lambda row: 1 if row['winner'] == row['fighter_1'] else 0, axis=1)


# Drop the specified columns
columns_to_drop = ['fight_id', 'fighter_1_id', 'fighter_1', 'fighter_2', 'fighter_2_id','decision_method','fight_duration_lastrnd','time_format','date', 'location','height_x','name_x','weight_x','height_y','name_y','weight_y' ]
data_cleaned = data.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned data
data_cleaned.head()

data_cleaned.to_csv("cleanedData2.csv", index = False)

In [3]:
#split up the data

winners = data_cleaned.iloc[:,0]
features = data_cleaned.iloc[:,1]


features = features.sample(frac =1 )



In [4]:
df = pd.read_csv("encoded_cleanedDataNEW.csv") # original dataset
##print(f"Initial shape: {df.shape}")

'''keep = np.array([4,2,1,6,10,11,12,37,38,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,104,105,130,132,133,134,135,136,137,138,139,140,141,142,143,144,145]) - 1
df = df.iloc[:,:] # interesting fields

df["date"] = pd.to_datetime(df["date"]) # date as datetime


df = df.replace(r'^\s*$', np.nan, regex=True) # replace empty string with nan
df = df.fillna(np.nan) # Fill empty and NaNs values with NaN

df = df.round(3) # format data

df.rename(columns={ # some renaming
    "B_win_by_KO/TKO": "B_win_by_KO_TKO",
    "R_win_by_KO/TKO": "R_win_by_KO_TKO"},inplace=True)
##print(f"shape: {df.shape}")

# considering matches with a winner as this will be a binary classification
drawRate = round(len(df["Winner"][df["Winner"]== "Draw"])/len(df.index)*100,3)
print(f"Draw rate is: {drawRate}%, converting to nan")
df["Winner"] = df["Winner"].replace("Draw", np.nan) # convert to nan to be dropped later
##df.dropna()'''



df

,winner,decision_method,fight_duration_lastrnd,fight_duration_lastrnd_time,time_format,weight_class,date,location,dob_x,height_x,...,win_percentage_x,win_percentage_y,reach_advantage,total_fights_x,total_fights_y,finishing_rate_x,finishing_rate_y,age_x,age_y,age_difference
0,0,3,3,292,1,2,331,92,835,9,...,0.800000,0.846154,-3.0,20.0,13.0,0.242500,0.406154,30,28,2
1,1,7,3,164,1,0,446,37,256,10,...,1.000000,0.714286,0.0,12.0,14.0,0.398333,0.094286,24,25,-1
2,0,4,3,292,1,58,355,126,612,7,...,0.588235,0.875000,0.0,17.0,8.0,0.211765,0.603750,31,25,6
3,1,7,3,173,1,0,380,78,286,7,...,0.875000,0.687500,4.0,16.0,16.0,0.196250,0.096250,27,29,-2
4,0,4,3,292,1,4,393,23,466,20,...,0.666667,0.739130,7.0,9.0,23.0,0.288889,0.269565,29,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3822,1,4,3,292,1,3,432,80,126,8,...,0.782609,0.620690,1.0,23.0,29.0,0.186087,0.092069,26,36,-10
3823,0,8,3,109,1,61,432,80,737,4,...,0.857143,0.588235,-3.0,14.0,17.0,0.430714,0.351765,27,34,-7
3824,1,4,3,292,1,58,432,80,606,11,...,0.687500,0.588235,3.0,16.0,17.0,0.393750,0.211765,31,31,0
3825,0,5,2,84,1,2,432,80,901,10,...,0.800000,0.666667,1.0,15.0,27.0,0.510667,0.187037,26,29,-3


In [5]:
##shuffling the data 
df_shuffled = df.sample(frac = 1)
df_shuffled.head(n=5)

,winner,decision_method,fight_duration_lastrnd,fight_duration_lastrnd_time,time_format,weight_class,date,location,dob_x,height_x,...,win_percentage_x,win_percentage_y,reach_advantage,total_fights_x,total_fights_y,finishing_rate_x,finishing_rate_y,age_x,age_y,age_difference
3323,0,7,1,275,1,6,225,30,48,3,...,0.605263,0.947368,5.0,38.0,19.0,0.070263,0.152632,35,28,7
1026,1,7,1,105,1,7,238,49,838,16,...,0.750000,0.695652,1.0,12.0,46.0,0.155000,0.083913,32,31,1
2591,1,5,3,27,1,57,94,139,614,13,...,0.734694,0.589744,-2.0,49.0,39.0,0.115102,0.137949,36,38,-2
2048,1,7,1,94,1,2,255,18,711,2,...,0.789474,0.625000,2.0,19.0,16.0,0.163158,0.153750,31,33,-2
2883,0,7,1,216,1,57,417,45,81,16,...,0.724138,0.707317,10.0,29.0,41.0,0.143448,0.104146,32,35,-3


In [6]:
##split up the data 
df_shuffled.columns

'''x = df_shuffled.drop([], axis = 1).values
y = df_shuffled["Winner"].values'''
y = df_shuffled.iloc[:,0]
x = df_shuffled.iloc[:,1]

In [7]:
import numpy as np

class CustomMinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    def fit(self, data):
        # Compute min and max values for each feature
        self.min_ = np.min(data, axis=0)
        self.max_ = np.max(data, axis=0)

    def transform(self, data):
        # Scale data to the range [0, 1]
        return (data - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = .5)

In [9]:
'''winners = y = df_shuffled["Winner"].values'''

scaler = CustomMinMaxScaler()

xTrainScaled = scaler.fit_transform(xTrain)
xTestScaled = scaler.fit_transform(xTest)



In [10]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(penalty = 'l1', tol = 1, solver = 'liblinear',
                            multi_class = "auto",
                            fit_intercept = False, 
                            max_iter =3)
model1.fit(xTrainScaled, yTrain)
joblib.dump(model1, 'model1.joblib')


ValueError: Expected 2D array, got 1D array instead:
array=[0.375 0.5   0.5   ... 0.875 0.375 0.875].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
testing_model = joblib.load("model1.joblib")
prediction = testing_model.predict(xTestScaled)
print(prediction)
for i in prediction:
    print(i)

ValueError: Expected 2D array, got 1D array instead:
array=[0.875 0.625 0.875 ... 0.5   0.375 0.5  ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
print("Accuracy score for Logistic Regression Model: %s" % (sklearn.metrics.accuracy_score(yTest,prediction)))
print("Precision score for logistic Regression Model: %s" % (sklearn.metrics.precision_score(yTest,prediction)))
print("Recall score for Logistic Regression Model: %s" % (sklearn.metrics.recall_score(yTest,prediction)))
print("F1 score for Logistic Regression Model: %s" % (sklearn.metrics.f1_score(yTest,prediction)))

NameError: name 'prediction' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier(n_neighbors = 4, p =3)
model2.fit(xTrainScaled,yTrain)
joblib.dump(model2, 'model2.joblib')

In [ ]:
knnModel = joblib.load("model2.joblib")
knnPrediction = knnModel.predict(xTestScaled)
for i in range(len(knnPrediction)): 
    print("Model 2 prediction based on data B for item: %d : %s" % (i + 1, knnPrediction[i]))

In [ ]:
knnModel_c = joblib.load("model2.joblib")
knnPrediction_c = knnModel_c.predict(xTestScaled)

C_answers = yTest
TP, FP = 0,0
#for i in range(len(knnPrediction_c)): 
    #print("Model 2 prediction based on data C: %s" % (knnPrediction_c[i]))
print("Accuracy Score for KNN Model: %s" %(sklearn.metrics.accuracy_score(C_answers,knnPrediction_c)))
print("Precision score for KNN Model: %s" %(sklearn.metrics.precision_score(C_answers,knnPrediction_c)))
print("Recall Score for KNN Model: %s" % (sklearn.metrics.recall_score(C_answers,knnPrediction_c)))
print("F1 Score for KNN Model: %s" % (sklearn.metrics.f1_score(C_answers,knnPrediction_c)))
FDR_knn = 1 - sklearn.metrics.precision_score(C_answers, knnPrediction_c)
print("False Detection Rate for KNN Model: %s" %(FDR_knn))
print("Matthews Coeffiecen for KNN Model: %s" % (sklearn.metrics.matthews_corrcoef(C_answers,knnPrediction_c)))

print(f'=====================================================================================================')

ValueError: Expected 2D array, got 1D array instead:
array=[0.875 0.625 0.875 ... 0.5   0.375 0.5  ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver = "lbfgs", alpha=.1, hidden_layer_sizes=(1,5))
model3.fit(xTrainScaled,yTrain)
joblib.dump(model3, 'model3.joblib')

In [ ]:
mlpModel = joblib.load("model3.joblib")
mlpPrediction = mlpModel.predict(xTestScaled)
for i in range(len(mlpPrediction)): 
    print("Model 4 prediction for item: %d : %s" % (i + 1, mlpPrediction[i]))

In [ ]:
#rand forest 
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(max_depth = 2)
model4.fit(xTrainScaled,yTrain)
joblib.dump(model4,'model4.joblib')


In [ ]:
forestModel = joblib.load("model4.joblib")
forestPrediction = forestModel.predict(xTestScaled)
for i in range(len(forestPrediction)): 
    print("Model 6 prediction for item: %d : %s" % (i + 1, forestPrediction[i]))



In [ ]:
forestModel_c = joblib.load("model4.joblib")
forestPrediction_c= forestModel_c.predict(xTestScaled)
C_answers = yTest
TP, FP = 0,0
for i in forestPrediction_c: 
    #print("Model 6 prediction for item: %s" % (forestPrediction_c[i]))
    if forestPrediction_c[i] == C_answers[i]: 
        TP += 1
        
    elif forestPrediction_c[i] == 1 and C_answers[i]==0 or forestPrediction_c[i]==0 and C_answers[i]==1:
        FP += 1
print("Accuracy Score for Forest Model: %s" %(sklearn.metrics.accuracy_score(C_answers,forestPrediction_c)))
print("Precision Score for Forest Model: %s" %(sklearn.metrics.precision_score(C_answers,forestPrediction_c)))
print("Recall Score for Forest Model: %s" % (sklearn.metrics.recall_score(C_answers,forestPrediction_c)))
print("F1 Score for Forest Model: %s" %(sklearn.metrics.f1_score(C_answers,forestPrediction_c)))
FDR_forest = 1 - sklearn.metrics.precision_score(C_answers,forestPrediction_c)
print("False Detection Rate for Forest Model: %s" %(FDR_forest))
print("Matthews Coefficent for Forest Model: %s" % (sklearn.metrics.matthews_corrcoef(C_answers,forestPrediction_c)))
print(f'=====================================================================================================')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Define the model
knn = KNeighborsClassifier()

# Set up the parameter grid
param_grid = {
    'n_neighbors': range(1, 21),
    'p': [1, 2, 3, 4, 5]  # You may add other values if you want to try custom distance metrics
}

# Set up Grid Search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(xTrainScaled, yTrain)

# Get the best model
best_model = grid_search.best_estimator_

# Save the optimized model
joblib.dump(best_model, 'optimized_model2.joblib')

# Print the best parameters and accuracy
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated accuracy:", grid_search.best_score_)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

# Train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(xTrainScaled, yTrain)

# Predict on the training or test set
y_pred_continuous = linear_model.predict(xTrainScaled)

# Apply threshold rounding to get binary predictions
y_pred_binary = np.where(y_pred_continuous >= 0.5, 1, 0)

# Evaluate accuracy
accuracy = accuracy_score(yTrain, y_pred_binary)
print("Accuracy (rounded to binary):", accuracy)

# Save the model if desired
joblib.dump(linear_model, 'binary_linear_model.joblib')


In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Assuming your data is preprocessed into X (features) and y (labels)

# Define the initial XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define a parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],   # Wider range for number of trees
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],  # Smaller learning rates
    'max_depth': [3, 5, 7, 9],  # Deeper trees for capturing complex patterns
    'subsample': [0.6, 0.8, 1.0],  # Experiment with different subsets of data
    'min_samples_split': [2, 5, 10, 20],  # Larger splits for stability
    'min_samples_leaf': [1, 2, 4, 10],  # Increase for regularization
    'max_features': ['sqrt', 'log2', None],  # Try limiting features for splits
}


# Perform Grid Search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=0,  # Suppress intermediate output
    n_jobs=-1
)

grid_search.fit(xTrain, yTrain)

# Extract the best parameters and accuracy
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(xTest)
accuracy = accuracy_score(yTest, y_pred)

# Print the final results
print("\nBest parameters found:", best_params)
print(f"Test Set Accuracy: {accuracy:.4f}")
